In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import math
import scipy.linalg
import  sys
from sklearn import preprocessing

In [3]:
df = pd.read_csv('ozone.csv')

FileNotFoundError: File b'ozone.csv' does not exist

In [286]:
df.head()

,maxO3,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,maxO3v,WindDirection
0,87.0,15.6,18.5,NaN,4.0,4.0,8.0,0.6946,-1.7101,-0.6946,84.0,North
1,82.0,NaN,NaN,NaN,5.0,5.0,7.0,-4.3301,-4.0000,-3.0000,87.0,North
2,92.0,15.3,17.6,19.5,2.0,NaN,NaN,2.9544,NaN,0.5209,82.0,East
3,114.0,16.2,19.7,NaN,1.0,1.0,0.0,NaN,0.3473,-0.1736,92.0,North
4,94.0,NaN,20.5,20.4,NaN,NaN,NaN,-0.5000,-2.9544,-4.3301,114.0,West


In [287]:
original_cols = df.columns

In [288]:
columns_missing  = {}
columns_missing = defaultdict(lambda : 0, columns_missing)
values = df.values

In [289]:
x = []
for i in range(len(values)):
    if i%10 == 0:
        values[i][-1] = np.nan

In [290]:
df = pd.DataFrame(values, columns =df.columns)

In [291]:
for col in df.columns:
    columns_missing[col] = df[col].isna().sum()

In [292]:
columns_missing
            

defaultdict(<function __main__.<lambda>()>,
            {'maxO3': 16,
             'T9': 37,
             'T12': 33,
             'T15': 37,
             'Ne9': 34,
             'Ne12': 42,
             'Ne15': 32,
             'Vx9': 18,
             'Vx12': 10,
             'Vx15': 21,
             'maxO3v': 12,
             'WindDirection': 12})

### One hot Encoding for the categorical variable wind direction


In [293]:
categories = df['WindDirection'].unique()
categories = list(categories)
categories.remove(np.nan)
for i in categories:
    if i is not np.nan:
        df[i] = 0

In [294]:
df.head()

,maxO3,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,maxO3v,WindDirection,North,East,West,South
0,87,15.6,18.5,NaN,4,4,8,0.6946,-1.7101,-0.6946,84,NaN,0,0,0,0
1,82,NaN,NaN,NaN,5,5,7,-4.3301,-4,-3,87,North,0,0,0,0
2,92,15.3,17.6,19.5,2,NaN,NaN,2.9544,NaN,0.5209,82,East,0,0,0,0
3,114,16.2,19.7,NaN,1,1,0,NaN,0.3473,-0.1736,92,North,0,0,0,0
4,94,NaN,20.5,20.4,NaN,NaN,NaN,-0.5,-2.9544,-4.3301,114,West,0,0,0,0


In [295]:
for index, direction in enumerate(df['WindDirection']):
    if direction is np.nan:
        for i in categories:
            df[i].iloc[index] = np.nan
    else:
        df[direction].iloc[index] = 1
        

C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [296]:
df.head()

,maxO3,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,maxO3v,WindDirection,North,East,West,South
0,87,15.6,18.5,NaN,4,4,8,0.6946,-1.7101,-0.6946,84,NaN,NaN,NaN,NaN,NaN
1,82,NaN,NaN,NaN,5,5,7,-4.3301,-4,-3,87,North,1.0,0.0,0.0,0.0
2,92,15.3,17.6,19.5,2,NaN,NaN,2.9544,NaN,0.5209,82,East,0.0,1.0,0.0,0.0
3,114,16.2,19.7,NaN,1,1,0,NaN,0.3473,-0.1736,92,North,1.0,0.0,0.0,0.0
4,94,NaN,20.5,20.4,NaN,NaN,NaN,-0.5,-2.9544,-4.3301,114,West,0.0,0.0,1.0,0.0


In [297]:
missing_flag = df.isna()
missing_flag = missing_flag.drop(columns=['WindDirection'])

### Initial imputation with mean for numerical variables and proportion for categorical variables

In [298]:
numerical = ['maxO3', 'T9', 'T12', 'T15', 'Ne9', 'Ne12', 'Ne15', 'Vx9', 'Vx12','Vx15', 'maxO3v']
categorical = ['North', 'East', 'West', 'South']

In [299]:
df.head()

,maxO3,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,maxO3v,WindDirection,North,East,West,South
0,87,15.6,18.5,NaN,4,4,8,0.6946,-1.7101,-0.6946,84,NaN,NaN,NaN,NaN,NaN
1,82,NaN,NaN,NaN,5,5,7,-4.3301,-4,-3,87,North,1.0,0.0,0.0,0.0
2,92,15.3,17.6,19.5,2,NaN,NaN,2.9544,NaN,0.5209,82,East,0.0,1.0,0.0,0.0
3,114,16.2,19.7,NaN,1,1,0,NaN,0.3473,-0.1736,92,North,1.0,0.0,0.0,0.0
4,94,NaN,20.5,20.4,NaN,NaN,NaN,-0.5,-2.9544,-4.3301,114,West,0.0,0.0,1.0,0.0


In [300]:
df.columns

Index(['maxO3', 'T9', 'T12', 'T15', 'Ne9', 'Ne12', 'Ne15', 'Vx9', 'Vx12',
       'Vx15', 'maxO3v', 'WindDirection', 'North', 'East', 'West', 'South'],
      dtype='object')

In [301]:
proportions = dict(df['WindDirection'].value_counts(normalize=True)) # replacing with proportions

proportions

{'West': 0.45, 'North': 0.24, 'South': 0.21, 'East': 0.1}

In [302]:
for col in numerical:
    df[col] = df[col].replace(np.nan,df[col].mean()) #initially replacing with means
for col in categorical:
    df[col] = df[col].replace(np.nan, proportions[col]) 


### Weight matrix ( 0 for missing values  and 1 for not missing)

In [303]:
weights = np.zeros((112,15))
for i in range(len(missing_flag.values)):
    for j in range(len(missing_flag.values[0])):
        if missing_flag.values[i][j] == False:
            weights[i][j] = 1


In [304]:
df.head


<bound method NDFrame.head of           maxO3         T9        T12        T15       Ne9      Ne12  Ne15  \
0     87.000000  15.600000  18.500000  22.405333  4.000000  4.000000   8.0   
1     82.000000  18.218667  21.462025  22.405333  5.000000  5.000000   7.0   
2     92.000000  15.300000  17.600000  19.500000  2.000000  4.985714   4.6   
3    114.000000  16.200000  19.700000  22.405333  1.000000  1.000000   0.0   
4     94.000000  18.218667  20.500000  20.400000  4.987179  4.985714   4.6   
5     80.000000  17.700000  19.800000  18.300000  6.000000  4.985714   7.0   
6     79.000000  16.800000  15.600000  14.900000  7.000000  8.000000   4.6   
7     79.000000  14.900000  17.500000  18.900000  5.000000  5.000000   4.6   
8    101.000000  16.100000  19.600000  21.400000  2.000000  4.985714   4.0   
9    106.000000  18.300000  21.462025  22.900000  5.000000  4.985714   4.6   
10   101.000000  17.300000  19.300000  20.200000  7.000000  7.000000   3.0   
11    90.000000  17.600000  20.300

In [305]:

scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(df.drop(columns = ['WindDirection']))
# scaled_df = pd.DataFrame(df.drop(columns = ['WindDirection']), columns=df.drop(columns = ['WindDirection']).columns)
# for col in scaled_df.columns:
#     scaled_df[col] = (scaled_df[col] - scaled_df[col].mean() / scaled_df[col].std())

U, sigma, V_transpose = np.linalg.svd(scaled_df) 
U.shape
sigma.shape
V_transpose.shape

(15, 15)

In [306]:
temp  =np.matmul( U[:,:15], np.diag(sigma)) 
X_hat = np.matmul(temp,  V_transpose)


In [307]:
X_hat

array([[-1.59087233e-01, -1.08146972e+00, -8.74021751e-01, ...,
         1.20825734e-15, -1.77986531e-15,  3.15709426e-16],
       [-3.46708539e-01, -1.29904522e-15, -2.17607827e-15, ...,
        -3.52766841e-01, -9.57268842e-01, -5.45638634e-01],
       [ 2.85340737e-02, -1.20536518e+00, -1.13958990e+00, ...,
         3.17490157e+00, -9.57268842e-01, -5.45638634e-01],
       ...,
       [ 2.91203903e-01, -1.11640197e-15,  3.88743083e-16, ...,
        -3.52766841e-01, -9.57268842e-01,  2.05264058e+00],
       [-3.09184278e-01, -1.92089109e-15, -2.09862430e-16, ...,
        -1.00337274e-16, -7.04909376e-16,  1.07610233e-15],
       [-7.96999675e-01, -1.04017123e+00, -8.44514179e-01, ...,
        -3.52766841e-01, -9.57268842e-01,  2.05264058e+00]])

In [308]:
X_new = np.multiply(weights,scaled_df) + np.multiply((1-weights), X_hat)
tolerance = ((X_hat - X_new)**2).mean(axis=None)
tolerance
    

1.5672032319925555e-30

In [310]:
X_new_unscaled = scaler.inverse_transform(X_new)


In [311]:
X_new_unscaled


array([[87.        , 15.6       , 18.5       , ...,  0.1       ,
         0.45      ,  0.21      ],
       [82.        , 18.21866667, 21.46202532, ...,  0.        ,
         0.        ,  0.        ],
       [92.        , 15.3       , 17.6       , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [99.        , 18.21866667, 21.46202532, ...,  0.        ,
         0.        ,  1.        ],
       [83.        , 18.21866667, 21.46202532, ...,  0.1       ,
         0.45      ,  0.21      ],
       [70.        , 15.7       , 18.6       , ...,  0.        ,
         0.        ,  1.        ]])

In [312]:
df.head()

,maxO3,T9,T12,T15,Ne9,Ne12,Ne15,Vx9,Vx12,Vx15,maxO3v,WindDirection,North,East,West,South
0,87.0,15.600000,18.500000,22.405333,4.000000,4.000000,8.0,0.694600,-1.710100,-0.6946,84.0,NaN,0.24,0.1,0.45,0.21
1,82.0,18.218667,21.462025,22.405333,5.000000,5.000000,7.0,-4.330100,-4.000000,-3.0000,87.0,North,1.00,0.0,0.00,0.00
2,92.0,15.300000,17.600000,19.500000,2.000000,4.985714,4.6,2.954400,-1.685325,0.5209,82.0,East,0.00,1.0,0.00,0.00
3,114.0,16.200000,19.700000,22.405333,1.000000,1.000000,0.0,-1.095754,0.347300,-0.1736,92.0,North,1.00,0.0,0.00,0.00
4,94.0,18.218667,20.500000,20.400000,4.987179,4.985714,4.6,-0.500000,-2.954400,-4.3301,114.0,West,0.00,0.0,1.00,0.00
